# Adaptively Sampled Distance Fields in Julia

This demonstrates basic usage of the AdaptiveDistanceFields package. This package implements the adaptive sampling described in: Sarah F. Frisken, Ronald N. Perry, and Thouis R. Jones. "Adaptively Sampled Distance Fields: A General Representation of Shape for Computer Graphics". SIGGRAPH 2000. 

In [1]:
using AdaptiveDistanceFields
using RegionTrees: allleaves, vertices
using StaticArrays: SVector
using Plots
gr()

Plots.GRBackend()

Let's define a simple curve consisting of a line and an arc:

In [16]:
plt = plot([0.35, 0.35], [0.2, 0.8], color=:black, 
    aspect_ratio=:equal, legend=nothing, xlim=(0, 1), ylim=(0, 1))
theta = linspace(-pi/2, pi/2)
plot!(plt, 0.3 * cos.(theta) .+ 0.35, 0.3 * sin.(theta) .+ 0.5, color=:black)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.0 
 
 
 0.2 
 
 
 0.4 
 
 
 0.6 
 
 
 0.8 
 
 
 1.0 
 
 
 0.0 
 
 
 0.2 
 
 
 0.4 
 
 
 0.6 
 
 
 0.8 
 
 
 1.0 
 
 
<polyline clip-path="url(#clip02)" style="stroke:#00002d; stroke-width:0.8; stroke-opacity:1; fill:none" points="
 256.181,308.749 263.505,308.514 270.798,307.81 278.031,306.641 285.175,305.011 292.199,302.926 299.076,300.395 305.776,297.429 312.272,294.04 318.538,290.242 
 324.548,286.05 330.277,281.481 335.701,276.555 340.798,271.291 345.548,265.712 349.93,259.84 353.928,253.699 357.523,247.315 360.703,240.713 363.452,233.921 
 365.761,226.967 367.62,219.88 369.02,212.687 369.957,205.42 370.427,198.108 370.427,190.781 369.957,183.469 369.02,176.201 367.62,169.009 365.761,161.922 
 363.452,154.968 360.703,148.176 357.523,141.574 353.928,135.19 349.93,129.049 345.548,123.177 340.798,117.597 335.701,112.334 330.277,107.408 324.548,102.839 
 318.538,98.6472 312.272,94.8488 305.776,91.4597 299.076,88.4937 292.199,85.9631 285.175,83.8783 278.031,82.2479 270.798,81.0784 263.505,80.3748 256.181,80.14 
 
 "/>

We can write down the distance from any point to that curve as a fairly simple function:

In [ ]:
function curve_distance(x)
    if x[2] < 0.2
        distance_to_line = norm(x - SVector(0.35, 0.2))
    elseif x[2] > 0.8
        distance_to_line = norm(x - SVector(0.35, 0.8))
    else
        distance_to_line = norm(x[1] - 0.35)
    end
    if x[1] >= 0.3
        distance_to_arc = norm(norm(x - SVector(0.35, 0.5)) - 0.3)
        return min(distance_to_arc, distance_to_line)
    else
        return distance_to_line
    end
end
        

Let's visualize that signed distance function:

In [26]:
x = linspace(0, 1, 101)
y = linspace(0, 1, 101)
plt = contour(x, y, (x, y) -> curve_distance(SVector(x, y)), 
    fill=true, legend=nothing, aspect_ratio=:equal)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.0 
 
 
 0.2 
 
 
 0.4 
 
 
 0.6 
 
 
 0.8 
 
 
 1.0 
 
 
 0.0 
 
 
 0.2 
 
 
 0.4 
 
 
 0.6 
 
 
 0.8 
 
 
 1.0

Now we can approximate that function with an adaptive sampling:

In [23]:
adf = AdaptiveDistanceField(curve_distance, SVector(0., 0), SVector(1., 1))

(::AdaptiveDistanceField) (generic function with 1 method)

And we can visualize the result. The adaptive sampling results in a finer grid near areas of sharp curvature:

In [24]:
x = linspace(0, 1)
y = linspace(0, 1)

# Plot the distance field
plt = contour(x, y, (x, y) -> adf(SVector(x, y)), fill=true, 
    legend=nothing, aspect_ratio=:equal)

# Draw each cell in the approximation
for leaf in allleaves(adf.root)
    v = hcat(collect(vertices(leaf.boundary))...)
    plot!(plt, v[1,[1,2,4,3,1]], v[2,[1,2,4,3,1]], color=:white)
end
plt

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.0 
 
 
 0.2 
 
 
 0.4 
 
 
 0.6 
 
 
 0.8 
 
 
 1.0 
 
 
 0.0 
 
 
 0.2 
 
 
 0.4 
 
 
 0.6 
 
 
 0.8 
 
 
 1.0